# EventAction

与 G4UserRunAction、G4UserSteppingAction 类似，G4UserEventAction 也是提取物理量的一个重要接口,主要通过 ndOfEventAction 函数来实现。该函数在每个 Event 之后之后调用一次，用户可以通过这个接口提取灵敏探测器的数据。

```cpp
class G4UserEventAction 
{
  public:

    G4UserEventAction();
    virtual ~G4UserEventAction() = default;
    virtual void SetEventManager(G4EventManager* value);

    virtual void BeginOfEventAction(const G4Event* anEvent);
    virtual void EndOfEventAction(const G4Event* anEvent);
      // Two virtual method the user can override.

    virtual void MergeSubEvent(G4Event* masterEvent, const G4Event* subEvent);
      // A virtual method to merge the results of a sub-event into the master
      // event. The ownership of "subEvent" and its contents blong to the
      // worker thread. 
      // Merging trajectories and scores are taken care by G4Event and
      // G4ScoringManager so the user does not need to take care of them.
      // But merging hits collections and UserEventInformation must be taken 
      // care by this method.
      // This method is invoked only for the case of sub-event parallelism.

  protected:

      G4EventManager* fpEventManager = nullptr; // not owned
};
```



以下是我们提供示例代码中的截图

![picevent1](img/wueventactionallhh.png)

在函数 **EndOfEventAction(const G4Event* anEvent)** 中，通过遍历灵敏探测器，获得每个灵敏探测器里面的数据。

- G4HCofThisEvent：G4Event 的一个成员对象，为一容器；多个 SD 的多个 HitCollection 对象储存在该容器中。


```cpp
void wuEventActionAll::EndOfEventAction(const G4Event* anEvent)
{
  // // 读取该事件SD中记录的数据
  G4HCofThisEvent *HCE = anEvent->GetHCofThisEvent();//SD  (hits collections of this event)
  if(HCE)
    {
      size_t nHCinHCE = HCE->GetCapacity();//SD个数
      // 
      for(size_t i = 0; i < nHCinHCE; i++)//遍历SD
	{
	  // 拿到HitCollection
	  wuHitCollection* HC = 0;
	  HC = (wuHitCollection*)(HCE->GetHC(i)); 
	  if(HC) OneHitOrder(HC);//写数据
	}
    }
}
```

这里数据筛选以及数据存储被封装到函数 **OneHitOrder**

![picevent2](img/onehitorder.png)

该函数遍历一个灵敏探测器的数据(step)，在存储数据之前，可先对数据进行简单的筛选，降低数据量。具体可参数 [StepingAction](StepingAction.html) 中的方式。


In [1]:
!date
!jupyter nbconvert EventAction.ipynb --to html

2024年 05月 01日 星期三 21:39:37 CST

[NbConvertApp] Converting notebook EventAction.ipynb to html


[NbConvertApp] Writing 567336 bytes to EventAction.html


